In [1]:
## IMPORTS ##
import pandas as pd
import numpy as np


## READ DATA ##
db1 = pd.read_csv('../Data/raw_databases/database.csv',encoding='ISO-8859-1')
db2 = pd.read_csv('../Data/raw_databases/unpub_db.csv')

C:\Users\aleja\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1,5,10,15,24,28,31,35,37,40,43,46,51,55,66,67,77,78,90,95,103,141,153,156,157,158) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
#classify into trait-specific data
eme_db1 = db1[db1['StandardisedTraitName'] == 'Specific Growth Rate']
eme_db2 = db2[db2['Trait'] == 'Specific Growth Rate']

comp_trait = ['Sulphide Production', 'Sulphide production', 'uMols methane','Fe2+ Oxidised', 'Caffeine Degradaded Per second (g)', 'Nitrate Production (mM)', 'SO4 Reduction Rate', 'H2S Production', 'mM Fe(II) production', 'Sulphur Oxidation Rate', 'Fe3+ Oxidation Rate', 'Fe2+ Oxidation Rate', 'Sulfate Reduction Rate', 'Nitrogen Removal Rate', 'Methanogenesis', 'Dehalogenation', 'CH4 produced']
comp_db1 = db1[db1['StandardisedTraitName'].isin(comp_trait)]
comp_db2 = db2[db2['Trait'] == 'Specific Respiration Rate']

#relabel columns
db1_col = {'OriginalID':'ID', 'StandardisedTraitValue':'TraitVal', 'StandardisedTraitUnit':'TraitUnit', 'ConPhylum':'Phylum', 'ConClass':'Class', 'ConOrder':'Order', 'ConFamily':'Family', 'ConGenus':'Genus', 'ConSpecies':'Species', 'ConTemp':'Temp', 'ConTempUnit':'TempUnit'}
eme_db1 = eme_db1.rename(columns=db1_col)
comp_db1 = comp_db1.rename(columns=db1_col)

db2_col = {'Strain':'ID', 'TraitValue':'TraitVal', 'TraitUnits':'TraitUnit', 'Temperature':'Temp', 'TemperatureUnits':'TempUnit'}
eme_db2 = eme_db2.rename(columns=db2_col)
comp_db2 = comp_db2.rename(columns=db2_col)

#Slice only important columns
col = list(db1_col.values())
list_db = [eme_db1, eme_db2, comp_db1, comp_db2]

eme_db1 = eme_db1[col]
eme_db2 = eme_db2[col]
comp_db1 = comp_db1[col]
comp_db2 = comp_db2[col]

In [14]:
#merge db1 and db2 for comp and eme
eme_db = pd.concat([eme_db1, eme_db2], axis=0)
comp_db = pd.concat([comp_db1, comp_db2], axis=0)

#adjust index column and add label column
eme_db['Index'] = eme_db['ID']        
for i in range(len(eme_db)):
    eme_db.iloc[i, -1] = i
eme_db = eme_db.set_index('Index')
eme_db['Label'] = 'Growth'


comp_db['Index'] = comp_db['ID']        
for i in range(len(comp_db)):
    comp_db.iloc[i, -1] = i
comp_db = comp_db.set_index('Index')
comp_db['Label'] = 'Respiration'

#remove any zero or negative value
eme_db = eme_db[eme_db['TraitVal'] > 0]
comp_db = comp_db[comp_db['TraitVal'] > 0]

#save as excel file
eme_db.to_excel('../Data/eme_db.xlsx')
comp_db.to_excel('../Data/comp_db.xlsx')